In [ ]:
# %cd drive/MyDrive/

/content/drive/MyDrive


In [1]:
!pip install transformers
# !pip install py-readability-metrics
!python -m nltk.downloader punkt

/home/dimion/miniconda3/envs/loora/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /home/dimion/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# # chat_history_ids = tokenizer.encode("Low level English text:" + tokenizer.eos_token, return_tensors='pt')
# # Let's chat for 5 lines
# for step in range(5):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     inp = input(">> User: ")
#     level = input("Response level: ")
#     new_user_input_ids = tokenizer.encode(inp + tokenizer.eos_token, return_tensors='pt')

#     level_ids = tokenizer.encode(level + tokenizer.eos_token, return_tensors='pt')
#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([level_ids, chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     # level_ids = tokenizer.encode(level + tokenizer.eos_token, return_tensors='pt')

#     # generated a response while limiting the total chat history to 1000 tokens, 
#     chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, num_beams=3, temperature=0.8, top_p=0.9)

#     lvl_ids = tokenizer.encode(level + tokenizer.eos_token, return_tensors='pt')
#     # import pdb; pdb.set_trace()
#     out = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

#     if (chat_history_ids[:,:level_ids.size(-1)] == level_ids).all():
#         chat_history_ids = chat_history_ids[:,level_ids.size(-1):]
#     # pretty print last ouput tokens from bot
#     print("DialoGPT: {}".format(out))

In [ ]:
# !git clone https://github.com/dimi1357/Loora-cond-bot.git

In [ ]:
%cd Loora-cond-bot/

/content/drive/MyDrive/Loora-cond-bot


In [ ]:
# !git clone https://github.com/cdimascio/py-readability-metrics.git

In [2]:
%cd py-readability-metrics/
!pip install .
%cd ..

/home/dimion/Loora-cond-bot/py-readability-metrics
Processing /home/dimion/Loora-cond-bot/py-readability-metrics
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for py-readability-metrics: filename=py_readability_metrics-1.4.4-py3-none-any.whl size=26783 sha256=f2efdc2bcd50a71e90ae77cbe339d7aec126e5535d3ae8633fd6ef480e49d222
  Stored in directory: /home/dimion/.cache/pip/wheels/2b/85/ab/93ce999a2e221be0498903aa07d68a9592a51dd161d127ae1c
Successfully built py-readability-metrics
  Attempting uninstall: py-readability-metrics
    Found existing installation: py-readability-metrics 1.4.4
    Uninstalling py-readability-metrics-1.4.

In [ ]:
# !python ./prepare_data.py data/combined_train.csv data/train.csv
# !python ./prepare_data.py data/combined_test.csv data/test.csv
# !python convert_to_signal.py data/train.csv data/test.csv easy medium hard

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, os
model_path = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
path = "dialog-model"
path = "temp_fix_model"
start = 0
if os.path.exists('index.torch'):
  print(f"loading checkpoint: {path}")
  start = torch.load("index.torch")
  model_path = path
model = AutoModelForCausalLM.from_pretrained(model_path)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
model.to(device)
model.train()

loading checkpoint: temp_fix_model


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [9]:
import csv
grades = set()
lines = []
first = True
with open("data/train_ready1.csv") as f:
  csvFile = csv.reader(f)
  contexts = []
  # targets = []
  inputs = []
  for line in csvFile:
    if first:
        first = False
        continue
    lines.append(line)
    grades.add(line[2])

In [7]:
import gc
from transformers import AdamW
import torch
import tqdm, sys, os

In [5]:
batch_size = 1
num_batches = len(lines) // batch_size
save_every = 1000

optimizer = AdamW(model.parameters(), lr=5e-5)

contexts = []
inputs = []
with tqdm.tqdm(desc="Training...", total=num_batches-(start // batch_size),
                      file=sys.stdout) as pbar:
  for i, line in enumerate(lines[start:]):
    while True:
      try:
        context, target, grade, *_ = line
        inputs.append(f"{grade} {context} {tokenizer.eos_token} {target} {tokenizer.eos_token}")
        contexts.append(f"{grade} {context} {tokenizer.eos_token}")
        # targets.append(target)
        if len(inputs) < batch_size:
          continue

        optimizer.zero_grad()
        context_dic = tokenizer.batch_encode_plus(contexts, padding='longest', return_tensors='pt', truncation=True)
        inputs_dic = tokenizer.batch_encode_plus(inputs, padding='longest', return_tensors='pt', truncation=True)

        mask = context_dic.attention_mask
        mask = torch.cat((mask, torch.zeros((batch_size, inputs_dic.attention_mask.size(-1)-mask.size(-1)), dtype=inputs_dic.attention_mask.dtype)), dim=-1)
        premise_mask = inputs_dic.attention_mask - mask
        labels = inputs_dic["input_ids"].clone() * (premise_mask==1) + -100 * ~(premise_mask==1)
#       import pdb; pdb.set_trace()
        inputs_dic = inputs_dic.to(device)
        labels = labels.to(device)
        outputs = model(**inputs_dic, labels=inputs_dic["input_ids"])
        inputs_dic, context_dic, inputs, contexts, mask = (None,) * 5
        loss = outputs.loss
        loss.backward()
        pbar.set_description(f"loss: {loss.item()}")
        pbar.update()
        contexts = []
        inputs = []
        if i+1 % save_every == 0:
          model.eval()
          model.save_pretrained(path)
          torch.save(start+i, 'index.torch')
          print(f"\nModel checkpoint is saved in: {path}")
          model.train()
        break
      except RuntimeError as e:
        gc.collect()
        inputs = []
        contexts = []

  model.eval()
  model.save_pretrained(path)
  torch.save(i, 'index.torch')
  print(f"\nModel checkpoint is saved in: {path}")


loss: 9.303303718566895: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████▉| 224408/224409 [4:21:16<00:00, 14.08it/s]
Model checkpoint is saved in: dialog-model
loss: 9.303303718566895: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 224409/224409 [4:21:22<00:00, 14.31it/s]


In [6]:
model.save_pretrained(path)

In [16]:
import csv
lines = []
first = True
with open("data/test_ready1.csv") as f:
  csvFile = csv.reader(f)
  contexts = []
  # targets = []
  inputs = []
  for line in csvFile:
    if first:
        first = False
        continue
    lines.append(line)

In [38]:
## Eval method 1
batch_size = 1
num_batches = len(lines) // batch_size
contexts = []
inputs = []
num_correct = 0

with tqdm.tqdm(desc="Running eval method 1...", total=len(lines),
                      file=sys.stdout) as pbar:
  for i, line in enumerate(lines):
    context, target, grade, *_ = line
    best = (10e5, None)
    for temp_grade in grades:
      inputs.append(f"{temp_grade} {context} {tokenizer.eos_token} {target} {tokenizer.eos_token}")
      contexts.append(f"{temp_grade} {context} {tokenizer.eos_token}")

      context_dic = tokenizer.batch_encode_plus(contexts, padding='longest', return_tensors='pt', truncation=True)
      inputs_dic = tokenizer.batch_encode_plus(inputs, padding='longest', return_tensors='pt', truncation=True)

      mask = context_dic.attention_mask
      mask = torch.cat((mask, torch.zeros((batch_size, inputs_dic.attention_mask.size(-1)-mask.size(-1)), dtype=inputs_dic.attention_mask.dtype)), dim=-1)
      premise_mask = inputs_dic.attention_mask - mask
      labels = inputs_dic["input_ids"].clone() * (premise_mask==1) + -100 * ~(premise_mask==1)
#       import pdb; pdb.set_trace()
      inputs_dic = inputs_dic.to(device)
      labels = labels.to(device)
      with torch.no_grad():
        outputs = model(**inputs_dic, labels=labels)
        loss = outputs.loss
      if loss < best[0]:
        best = (loss, temp_grade)
      inputs_dic, context_dic, inputs, contexts, mask, labels = None, None, [], [], None, None
    if grade == best[-1]:
      num_correct += 1
    pbar.set_description(f"loss: {loss.item()}, running accuracy: {float(num_correct) / (i+1)}")
    pbar.update()
    contexts = []
    inputs = []

print(f"The model accuracy is: {float(num_correct) / len(lines)}, using eval method 1")

loss: 5.997912406921387, running accuracy: 0.5777793046585131: 100%|██████████████████████████████████████████████████████████████████| 14554/14554 [15:00<00:00, 16.16it/s]
The model accuracy is: 0.5777793046585131, using eval method 1


In [45]:
## Eval method 2
import gc
from transformers import AdamW
import torch
import tqdm, sys, os
from readability import Readability

contexts = []
inputs = []
num_correct = 0
thresholds = [2.0, 6.0, 10e8]
signals = ["easy", "medium", "hard"]

with tqdm.tqdm(desc="Running eval method 2...", total=len(lines),
                      file=sys.stdout) as pbar:
  for i, line in enumerate(lines):
    context, target, grade, *_ = line
    contexts.append(f"{grade} {context} {tokenizer.eos_token}")

    context_dic = tokenizer.batch_encode_plus(contexts, padding='longest', return_tensors='pt', truncation=True)
    context_dic = context_dic.to(device)
    context_ids = context_dic["input_ids"]
    response_ids = model.generate(context_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    out = tokenizer.decode(response_ids[:, context_ids.shape[-1]:][0], skip_special_tokens=True)
    try:
        r = Readability(out)
        score = r.flesch_kincaid()
        pred_grade = score.grade_level
#         import pdb; pdb.set_trace()
        for j in range(len(thresholds)):
          if float(pred_grade) < thresholds[j]:
            pred_grade = signals[j]
            break

    except ZeroDivisionError:
        pred_grade = ""
    if grade == pred_grade:
      num_correct += 1
    
    pbar.update()
    pbar.set_description(f"running accuracy: {float(num_correct) / (i+1)}")
    contexts = []
    inputs = []

print(f"The model accuracy is: {float(num_correct) / len(lines)}, using eval method 2")

running accuracy: 0.7763501442902295: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 14554/14554 [1:17:34<00:00,  3.13it/s]
The model accuracy is: 0.7763501442902295, using eval method 2


In [ ]:
# chat_history_ids = tokenizer.encode("Low level English text:" + tokenizer.eos_token, return_tensors='pt')
# Let's chat for 5 lines
for step in range(10**8):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    inp = input(">> User: ")
    level = input("Response level: ")
    new_user_input_ids = tokenizer.encode(inp + tokenizer.eos_token, return_tensors='pt')

    level_ids = tokenizer.encode(level + tokenizer.eos_token, return_tensors='pt')
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([level_ids, chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # level_ids = tokenizer.encode(level + tokenizer.eos_token, return_tensors='pt')

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, num_beams=3, temperature=0.8, top_p=0.9)

    lvl_ids = tokenizer.encode(level + tokenizer.eos_token, return_tensors='pt')
    # import pdb; pdb.set_trace()
    out = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    if (chat_history_ids[:,:level_ids.size(-1)] == level_ids).all():
        chat_history_ids = chat_history_ids[:,level_ids.size(-1):]
    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(out))